In [1]:
# https://hotspot.readthedocs.io/en/latest/Lineage_Tutorial.html
# https://colab.research.google.com/drive/1UArV6s-3JN2nKuHcsjfsYu33elR1ySa6#scrollTo=Dj24KOME2K6l

import sys
import warnings; warnings.simplefilter('ignore')

import hotspot
import scanpy as sc

import numpy as np
import mplscience

import anndata as ad

In [2]:
adata = ad.read_h5ad("/home/users/kzlin/kzlinlab/projects/scContrastiveLearn/out/kevin/Writeup5/Larry_41093_2000_norm_log.h5ad")
adata

AnnData object with n_obs × n_vars = 41093 × 2000
    obs: 'Library', 'Cell barcode', 'time_info', 'Starting population', 'state_info', 'Well', 'SPRING-x', 'SPRING-y', 'Time point', 'clone_id', 'n_genes'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'data_des', 'hvg', 'log1p'
    obsm: 'X_clone', 'X_emb'
    layers: 'ambiguous', 'matrix', 'raw_counts', 'spliced', 'unspliced'

In [3]:
scVI_embedding = np.load("/home/users/kzlin/kzlinlab/projects/scContrastiveLearn/out/kevin/Writeup5/Larry_scvi_full_embeddings.npy")
scVI_embedding.shape

(41093, 10)

In [4]:
adata.obsm["X_scVI"] = scVI_embedding

# Verify the assignment
print(adata.obsm["X_scVI"].shape)

NameError: name 'scCL_embedding' is not defined